This is a small modification and combination of these two notebooks
1. https://www.kaggle.com/takamotoki/lgbm-iii-part2 
2. https://www.kaggle.com/code1110/riiid-lgb-hyperparameter-tuning
Hope that it will help you guys...

# ## Reading Data and Importing Libraries ##

In [ ]:
import riiideducation
# import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
env = riiideducation.make_env()

In [ ]:
nrows = 100 * 10000

In [ ]:
train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                   usecols=[1, 2, 3, 4, 5, 7, 8, 9],
                   #nrows = nrows,
                   dtype={'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'task_container_id': 'int16',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )

In [ ]:
train.head(10)

In [ ]:
#reading in question df
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',                         
                            usecols=[0, 3],
                            dtype={'question_id': 'int16',
                              'part': 'int8'}
                          )

Affirmatives (True) for content_type_id are only for those with a different type of content (lectures). These are not real questions.

In [ ]:
questions_df.head(10)

In [ ]:
#removing True or 1 for content_type_id

train = train[train.content_type_id == False].sort_values('timestamp').reset_index(drop = True)
elapsed_mean = train.prior_question_elapsed_time.mean()

# Extract Feature

In [ ]:
# feature_1
results_c_final = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean','sum','count','std'])
results_c_final.columns = ["content_y_mean","content_y_sum","content_y_count","content_y_std"]

# feature_2
results_u_final = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum', 'count','std'])
results_u_final.columns = ['user_y_mean', 'user_y_sum', 'user_y_count','user_y_std']

# feature_3
results_ct_final = train[['content_id','prior_question_elapsed_time']].groupby(['content_id']).agg(['mean', 'sum', 'count','std'])
results_ct_final.columns = ['content_t_mean', 'content_t_sum', 'content_t_count','content_t_std']

# feature_4
results_ut_final = train[['user_id','prior_question_elapsed_time']].groupby(['user_id']).agg(['mean', 'sum', 'count','std'])
results_ut_final.columns = ['user_t_mean', 'user_t_sum', 'user_t_count','user_t_std']

In [ ]:
# feature_5
group1 = train.loc[(train.content_type_id == False), ['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['count'])
group1.columns = ['avg_questions']

group2 = train.loc[(train.content_type_id == False), ['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['nunique'])
group2.columns = ['avg_questions']

group3 = group1 / group2
group3['avg_questions_seen'] = group3.avg_questions.cumsum()

In [ ]:
train = pd.merge(train, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')

In [ ]:
results_q_final = train.loc[train.content_type_id == False, ['question_id','answered_correctly']].groupby(['question_id']).agg(['mean'])
results_q_final.columns = ['quest_pct']

In [ ]:
results_q2_final = train.loc[train.content_type_id == False, ['question_id','part']].groupby(['question_id']).agg(['count'])
results_q2_final.columns = ['count']

In [ ]:
# feature_6
question2 = pd.merge(questions_df, results_q_final, left_on = 'question_id', right_on = 'question_id', how = 'left')

In [ ]:
question2 = pd.merge(question2, results_q2_final, left_on = 'question_id', right_on = 'question_id', how = 'left')

In [ ]:
question2.quest_pct = round(question2.quest_pct,5)

In [ ]:
train.drop(['timestamp', 'content_type_id', 'question_id', 'part'], axis=1, inplace=True)

In [ ]:
results_c_tv = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean','sum','count','std'])
results_c_tv.columns = ["content_y_mean","content_y_sum","content_y_count","content_y_std"]

results_u_tv = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum', 'count','std'])
results_u_tv.columns = ['user_y_mean', 'user_y_sum', 'user_y_count','user_y_std']

results_ct_tv = train[['content_id','prior_question_elapsed_time']].groupby(['content_id']).agg(['mean', 'sum', 'count','std'])
results_ct_tv.columns = ['content_t_mean', 'content_t_sum', 'content_t_count','content_t_std']

results_ut_tv = train[['user_id','prior_question_elapsed_time']].groupby(['user_id']).agg(['mean', 'sum', 'count','std'])
results_ut_tv.columns = ['user_t_mean', 'user_t_sum', 'user_t_count','user_t_std']

prior_mean_user = results_ut_tv.user_t_mean.mean()

# Validation Set

In [ ]:
validation = pd.DataFrame()

for i in range(6):
    last_records = train.drop_duplicates('user_id', keep = 'last')
    train = train[~train.index.isin(last_records.index)]
    validation = validation.append(last_records)
    #print('validation : ', i)


X = pd.DataFrame()

for i in range(30):
    last_records = train.drop_duplicates('user_id', keep = 'last')
    train = train[~train.index.isin(last_records.index)]
    X = X.append(last_records)
    #print('X : ', i)


In [ ]:
del(train)

In [ ]:
X = pd.merge(X, results_u_tv, on=['user_id'], how="left")
X = pd.merge(X, results_c_tv, on=['content_id'], how="left")
X = pd.merge(X, results_ut_tv, on=['user_id'], how="left")
X = pd.merge(X, results_ct_tv, on=['content_id'], how="left")
X = pd.merge(X, group3, left_on=['task_container_id'], right_index= True, how="left")

In [ ]:
validation = pd.merge(validation, results_u_tv, on=['user_id'], how="left")
validation = pd.merge(validation, results_c_tv, on=['content_id'], how="left")
validation = pd.merge(validation, results_ut_tv, on=['user_id'], how="left")
validation = pd.merge(validation, results_ct_tv, on=['content_id'], how="left")
validation = pd.merge(validation, group3, left_on=['task_container_id'], right_index= True, how="left")

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

X.prior_question_had_explanation.fillna(False, inplace = True)
validation.prior_question_had_explanation.fillna(False, inplace = True)

validation["prior_question_had_explanation_enc"] = lb_make.fit_transform(validation["prior_question_had_explanation"])
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])

In [ ]:
content_mean = question2.quest_pct.mean()

question2.quest_pct.mean()
#there are a lot of high percentage questions, should use median instead?

In [ ]:
#filling questions with no info with a new value
question2.quest_pct = question2.quest_pct.mask((question2['count'] < 3), .65)


#filling very hard new questions with a more reasonable value
question2.quest_pct = question2.quest_pct.mask((question2.quest_pct < .2) & (question2['count'] < 21), .2)

#filling very easy new questions with a more reasonable value
question2.quest_pct = question2.quest_pct.mask((question2.quest_pct > .95) & (question2['count'] < 21), .95)

In [ ]:
X = pd.merge(X, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
validation = pd.merge(validation, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
X.part = X.part - 1
validation.part = validation.part - 1

In [ ]:
y = X['answered_correctly']
X = X.drop(['answered_correctly'], axis=1)
X.head()

y_val = validation['answered_correctly']
X_val = validation.drop(['answered_correctly'], axis=1)

In [ ]:
X.info()

# training

In [ ]:
columns_features = [c for c in X.columns if c not in ['user_id','content_id','prior_question_had_explanation']]

In [ ]:
X = X[columns_features]
X_val = X_val[columns_features]

X.fillna(-1,  inplace=True)
X_val.fillna(-1, inplace=True)

In [ ]:
import lightgbm as lgb

y_true = np.array(y_val)
lgb_train = lgb.Dataset(X, y, categorical_feature = ['part', 'prior_question_had_explanation_enc'],free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, categorical_feature = ['part', 'prior_question_had_explanation_enc'], reference=lgb_train, free_raw_data=False)

In [ ]:
def objective(trial):    
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 32, 512),
            'boosting_type': 'gbdt',
            'max_bin': trial.suggest_int('max_bin', 700, 900),
            'objective': 'binary',
            'metric': 'auc',
            'max_depth': trial.suggest_int('max_depth', 4, 16),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 16),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 8),
            'min_child_samples': trial.suggest_int('min_child_samples', 4, 80),
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
            'early_stopping_rounds': 10
            }
    model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=1000, num_boost_round=1300)
    val_pred = model.predict(X_val)
    score = roc_auc_score(y_true, val_pred)
    print(f"AUC = {score}")
    return score

In [ ]:
import optuna

In [ ]:
# Bayesian optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

In [ ]:
print('Number of finished trials: {}'.format(len(study.trials)))

print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

In [ ]:
# plot history
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

In [ ]:
model = lgb.train(trial.params, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=1000,num_boost_round=1300)

In [ ]:
y_pred = model.predict(X_val)
roc_auc_score(y_true, y_pred)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#displaying the most important features by split
lgb.plot_importance(model)
plt.show()

In [ ]:
#displaying the most important features by gain
lgb.plot_importance(model, importance_type = 'gain')
plt.show()

## Making Predictions for New Data ##

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df['task_container_id'] = test_df.task_container_id.mask(test_df.task_container_id > 9999, 9999)
    test_df = pd.merge(test_df, group3, left_on=['task_container_id'], right_index= True, how="left")
    test_df = pd.merge(test_df, question2, left_on = 'content_id', right_on = 'question_id', how = 'left')
    
    test_df = pd.merge(test_df, results_u_final, on=['user_id'],  how="left")
    test_df = pd.merge(test_df, results_c_final, on=['content_id'],  how="left")
    test_df = pd.merge(test_df, results_ut_final, on=['user_id'],  how="left")
    test_df = pd.merge(test_df, results_ct_final, on=['content_id'],  how="left")
    
    
    test_df['quest_pct'].fillna(content_mean, inplace=True)
    test_df['part'] = test_df.part - 1
    test_df['part'].fillna(4, inplace = True)
    
    test_df['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
    test_df['prior_question_had_explanation'].fillna(False, inplace=True)
    test_df["prior_question_had_explanation_enc"] = lb_make.fit_transform(test_df["prior_question_had_explanation"])
    
   
    
    X_test = test_df[columns_features]
    X_test.fillna(-1, inplace=True)

    test_df['answered_correctly'] =  model.predict(X_test)
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])